# Processing digital elevation data for deep learning models using Keras Spatial 

Aiman Soliman, Jeffrey Terstriep \
NCSA, University of Illinois Urbana, Illinois

 

## 1. How to Avoid adhoc preprocessing with KS

- background problem adhoc preprocessing 
- introduce our solution (purpose of this notebook)
- what the readers going to learn from this notebook (objectives)

## 2. What is Keras Spatial?

In [4]:
# rational behind keras spatial 
# why keras?
# KS design 
# introduce the different components 


In [1]:
import fiona
import shapely 
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### 2.1 Spatial Data Generator

If you are familiar with the deep learning framework Keras, then the SpatialDataGenerator (SDG) resambles the standard Keras ImageDataGenerator class [1 F. Chollet. 2016]. The main difference is that SDG extracts sample data from a single large raster file. This setting is more convenient for remote sensing applications, where the dimensions of an input image file are not equal to the dimensions of a single sample. For example you will find often that a raster file dimiensions are far bigger than the sample dimensions that you want to feed to a deep learning model. In other words a single raster file is likely to contian a large number of samples.

It should be also mentioned that the SDG class does not inharite the flow and flow_from_directory methods provided by the standard ImageDataGenerator to acount for the preprocessing steps that are needed for extracting multiple samples from a single raster. 

In [13]:
from keras_spatial.datagen import SpatialDataGenerator

raster_path = 'data/cl1.tif'
    
sdg = SpatialDataGenerator() 
sdg.source = raster_path

Notice that creating an SDG instance requires a raster data source and an array size. The raster source can be specified as an argument to the SDG initialization or as an attribute. The raster is opened immediately and the raster metadata (band count, CRS, pixel resolution, etc.) is available using the SDG src attribute. If the raster contains multiple bands, all bands are read by default. The indexes attribute can be set to specify the desired bands. 

The array size should match that of the model receptive layer. It is set using the width and height attributes on the SDG instance or as arguments to the flow_from_dataframe with flow_from_dataframe arguments taking precedence. The SDG supports standard interpolation methods (nearest neighbor, bilinear, cubic, etc) to handle resampling from sample size to array size. 

The SDG leverages a warped virtual raster class from Python’s excellent Rasterio package [gillies2013rasterio] which provides access to any raster format supported by the GDAL library, access to local raster and remote rasters over HTTP, and automatic reprojection and resampling to ensure data is registered spatially regardless the original raster projection and pixel resolution.



### 2.2 Sample Generator Utilities

The Keras Spatial grid module provides two functions for generating GeoDataFrames that define the sample boundaries, regular_grid, and random_grid. Both require the spatial extent of the study area, the sample size in coordinate system units and the coordinate reference system (CRS). The regular_grid may also include an overlap that increases the number of samples available to model.

The SDG class includes methods that provide a shortcut in accessing these functions where the spatial extent and CRS are determined directly from the raster source. In this case, the sample width and height are specified in pixels and the sample size is automatically determined by multiplying the raster resolution. Regardless of the method used to create the geodataframe, the sample boundaries are specified in spatial coordinates and are unrelated to the final array size that will be passed to the model.


In [ ]:
df_regular = sdg.regular_grid(100, 100)
print(f'Created {len(df)} samples')
df.head()

### 2.3 Callback Functions 

In [7]:
# explain and give a generic example?

## 3. A practical example: Preprocessing DEM 

In [8]:
# examples to demonstrate the flexiable design of KS

### 3.1 Feeding data directly to a deep learning model 

In [14]:
# example of Feeding samples to a DL model 

# train_gen = zip(labels.flow_from_dataframe(df), patches.flow_from_dataframe(df))
# model(train_gen)


### 3.2 managing sample space 

The GeoDataFrame can also be used to perform sample filtering and determining the global attributes of the sample set. For the former, the study area may be an irregular boundary within the spatial extent. The Keras Spatial grid module contains a mask function to remove samples outside the study area based on an arbitrary polygonal boundary. 

As an example of the latter, the maximum value across all the samples may be required to normalize samples to a known range. Using the SDG, each sample is extracted, it’s maximumly calculated and stored in a column of the GeoDataFrame. Later, the column is used to determine the maximum across all samples. An example of this technique is shown in section 4, where the sample set min/max is passed to a normalization callback.

In [11]:
# selecting samples with specific attributes (e.g., number of features pixels)  
# selecting samples within a watershed 

### 3.3 Estimating global statistics and batch normalization

In [ ]:
# 

### 3.4 Calculating DEM derivative on the fly  

In [12]:
# Estimating global statistics for sample normalization



## 4. Accounting for Samples Bias using Kears Spatial 

In [ ]:
# class imblance and using soil/geomorphology map to understand the 

## 5. Conclusions